In [ ]:
# !pip install transformers datasets torchaudio jiwer
# !pip install datasets==3.6.0

In [ ]:
import os
import torch
from datasets import load_dataset, Audio
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC, Trainer, TrainingArguments

torch.manual_seed(42)

In [ ]:
from huggingface_hub import login

login("")

In [ ]:
# Load dataset
dataset = load_dataset("mozilla-foundation/common_voice_17_0", "el", split="train")
test_dataset = load_dataset("mozilla-foundation/common_voice_17_0", "el", split="test")

In [ ]:
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))
test_dataset = test_dataset.cast_column("audio", Audio(sampling_rate=16000))

In [ ]:
dataset.column_names

In [ ]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
MODEL_ID = "jonatasgrosman/wav2vec2-large-xlsr-53-greek"
processor = Wav2Vec2Processor.from_pretrained(MODEL_ID)
model = Wav2Vec2ForCTC.from_pretrained(MODEL_ID)
def preprocess(batch):
    audio = batch["audio"]
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["labels"] = processor.tokenizer(batch["sentence"]).input_ids
    return batch

In [ ]:
columns_to_remove = [col for col in dataset.column_names]

In [ ]:
dataset = dataset.map(preprocess, remove_columns=columns_to_remove)
test_dataset = test_dataset.map(preprocess, remove_columns=columns_to_remove)

In [ ]:
print(dataset)
print(test_dataset)

In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch


In [ ]:
# # Define metrics
# wer_metric = load_metric("wer")
# def compute_metrics(pred):
#     pred_logits = pred.predictions
#     pred_ids = torch.argmax(torch.tensor(pred_logits), dim=-1)
#     pred_str = processor.batch_decode(pred_ids)
#     label_str = processor.batch_decode(pred.label_ids, group_tokens=False)
#     wer = wer_metric.compute(predictions=pred_str, references=label_str)
#     return {"wer": wer}

In [ ]:
dataset

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)
# def data_collator(features):
#     input_values = [f["input_values"] for f in features]
#     labels = [f["labels"] for f in features]
#     batch = processor.pad({"input_values": input_values, "labels": labels}, return_tensors="pt")
#     batch["labels"] = torch.tensor([[-100 if token == processor.tokenizer.pad_token_id else token for token in label]
#                                     for label in batch["labels"]])


In [ ]:
training_args = TrainingArguments(
   output_dir="./test",
   per_device_train_batch_size=8,
   per_device_eval_batch_size=8,
  #  eval_strategy="steps",
  #  eval_steps=10,                 # Валидация каждые 10 шагов
   save_steps=10,                 # Сохранение каждые 10 шагов
   logging_steps=1,              # Логи каждые 10 шагов
   max_steps=100,                 # Ровно 100 шагов
   learning_rate=1e-6,
   weight_decay=0.005,
   warmup_steps=20,               # 20% от 100 шагов
   save_total_limit=2,
   metric_for_best_model="eval_loss",
   greater_is_better=False,       # Меньший loss = лучше
)

In [ ]:
# Create Trainer instance
trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    # compute_metrics=compute_metrics,
    train_dataset=dataset,
    # eval_dataset=test_dataset,
    tokenizer=processor.feature_extractor,
)

In [ ]:
# Start training
trainer.train()